In [1]:
from source.render import render
render(4)

## 3. Interpolation

[Study Notes](#study-notes)

[Questions](#questions)



### Key Points

##### 1. 🧩 Cubic Spline Definition  
- A cubic spline interpolant $S$ consists of piecewise cubic polynomials $S_j(x)$ defined on intervals $[x_j, x_{j+1}]$.  
- Each cubic polynomial has 4 coefficients: $a_j, b_j, c_j, d_j$.  
- For $n+1$ data points, there are $n$ cubic polynomials and $4n$ unknown coefficients.

##### 2. 📏 Conditions for Cubic Splines  
- $S_j(x_j) = f(x_j)$ and $S_j(x_{j+1}) = f(x_{j+1})$ (interpolates data points).  
- $S_j(x_{j+1}) = S_{j+1}(x_{j+1})$ (continuity of function at interior nodes).  
- $S_j'(x_{j+1}) = S_{j+1}'(x_{j+1})$ (continuous first derivative at interior nodes).  
- $S_j''(x_{j+1}) = S_{j+1}''(x_{j+1})$ (continuous second derivative at interior nodes).  
- Boundary conditions must be specified to uniquely determine the spline.

##### 3. 🛠 Boundary Conditions  
- **Natural spline:** $S''(x_0) = 0$ and $S''(x_n) = 0$.  
- **Clamped spline:** $S'(x_0) = f'(x_0)$ and $S'(x_n) = f'(x_n)$.  
- Natural splines approximate the shape of a flexible rod passing through points.  
- Clamped splines generally provide more accurate approximations if endpoint derivatives are known.

##### 4. 🔢 Number of Equations and Unknowns  
- Total unknowns: $4n$ coefficients for $n$ cubic polynomials.  
- Total equations: $2n$ from interpolation + $n-1$ from first derivative continuity + $n-1$ from second derivative continuity + 2 from boundary conditions = $4n$.

##### 5. 🧮 Key Linear System for Second Derivatives  
- The second derivatives $c_j$ satisfy the tridiagonal system:

$$
h_{j-1} c_{j-1} + 2(h_{j-1} + h_j) c_j + h_j c_{j+1} = 3 \left( \frac{a_{j+1} - a_j}{h_j} - \frac{a_j - a_{j-1}}{h_{j-1}} \right), \quad j=1, \ldots, n-1
$$

- $h_j = x_{j+1} - x_j$, $a_j = f(x_j)$.  
- For natural splines, $c_0 = 0$ and $c_n = 0$.  
- For clamped splines, boundary conditions modify the first and last equations.

##### 6. ✅ Existence and Uniqueness  
- The coefficient matrix of the linear system is strictly diagonally dominant, ensuring a unique solution.  
- Unique natural spline exists for any set of nodes and function values.  
- Unique clamped spline exists if endpoint derivatives are known.

##### 7. 🧪 Example Data for Clamped Spline  
- Given points $(0,1), (1,e), (2,e^2), (3,e^3)$ and derivatives $f'(0)=1$, $f'(3)=e^3$, the clamped spline can be constructed by solving the linear system for $c_j$.

##### 8. 📉 Error Bound for Cubic Splines  
- If $f \in C^4[a,b]$ and $M = \max |f^{(4)}(x)|$, then for clamped spline $S$:

$$
|f(x) - S(x)| \leq \frac{5}{384} M h^4, \quad h = \max_j (x_{j+1} - x_j)
$$

- Error decreases as the fourth power of the maximum subinterval length $h$.  
- A similar fourth-order error bound exists for natural splines but is more complex to express.



<br>

## Study Notes

### 1. 🧩 What is Interpolation and Why Use Cubic Splines?

**Interpolation** is a method used to estimate values between known data points. Imagine you have a set of points on a graph, and you want to draw a smooth curve that passes exactly through all these points. This curve is called an **interpolating function**.

**Cubic spline interpolation** is a popular technique where the interpolating function is made up of several cubic polynomials, each defined on a small interval between two consecutive data points. Instead of fitting one big polynomial to all points (which can oscillate wildly), cubic splines fit smaller cubic polynomials piecewise, ensuring smoothness and continuity.

Why cubic splines? Because they provide a smooth curve that not only passes through the data points but also has continuous first and second derivatives, making the curve look natural and smooth—like a flexible rod bent through the points.


### 2. 🧮 Defining the Cubic Spline Interpolant

Suppose you have **n + 1** data points:

$$
(x_0, y_0), (x_1, y_1), \ldots, (x_n, y_n)
$$

where $a = x_0 < x_1 < \cdots < x_n = b$.

A **cubic spline interpolant** $S(x)$ is a function made up of $n$ cubic polynomials $S_j(x)$, each defined on the interval $[x_j, x_{j+1}]$ for $j = 0, 1, \ldots, n-1$.

Each cubic polynomial has the form:

$$
S_j(x) = a_j (x - x_j)^3 + b_j (x - x_j)^2 + c_j (x - x_j) + d_j
$$

or equivalently:

$$
S_j(x) = a_j x^3 + b_j x^2 + c_j x + d_j
$$

but the shifted form (using $x - x_j$) is often more convenient.

##### Conditions the spline must satisfy:

1. **Interpolation condition:** The spline passes through the data points:

$$
S_j(x_j) = y_j, \quad S_j(x_{j+1}) = y_{j+1}
$$

2. **Continuity at interior points:** The spline pieces join smoothly:

$$
S_j(x_{j+1}) = S_{j+1}(x_{j+1}) \quad \text{for } j = 0, \ldots, n-2
$$

3. **Continuous first derivative:** The slope is continuous at interior points:

$$
S_j'(x_{j+1}) = S_{j+1}'(x_{j+1}) \quad \text{for } j = 0, \ldots, n-2
$$

4. **Continuous second derivative:** The curvature is continuous at interior points:

$$
S_j''(x_{j+1}) = S_{j+1}''(x_{j+1}) \quad \text{for } j = 0, \ldots, n-2
$$

5. **Boundary conditions:** To uniquely determine the spline, we need two additional conditions at the endpoints $x_0$ and $x_n$. Common choices are:

- **Natural (or free) boundary conditions:**

$$
S''(x_0) = 0, \quad S''(x_n) = 0
$$

This means the spline is "relaxed" at the ends, like a free bending rod.

- **Clamped boundary conditions:**

$$
S'(x_0) = f'(x_0), \quad S'(x_n) = f'(x_n)
$$

Here, the slope at the endpoints is fixed to match the derivative of the underlying function (if known).


### 3. 🔢 How Many Unknowns and Equations?

For $n$ intervals, there are $n$ cubic polynomials, each with 4 coefficients:

$$
4n \quad \text{unknown coefficients}
$$

We need $4n$ independent equations to solve for these coefficients.

- **Interpolation at nodes:** $2n$ equations (each polynomial passes through two points).
- **Continuity of function at interior points:** $n-1$ equations (already implied by interpolation).
- **Continuity of first derivative at interior points:** $n-1$ equations.
- **Continuity of second derivative at interior points:** $n-1$ equations.
- **Boundary conditions:** 2 equations.

Adding these up:

$$
2n + (n-1) + (n-1) + 2 = 4n
$$

which matches the number of unknowns.


### 4. 🛠 Constructing a Natural Cubic Spline: Step-by-Step

Let's walk through the process of constructing a **natural cubic spline** for a simple example with three points:

$$
(1, 2), (2, 3), (3, 5)
$$

##### Step 1: Define the cubic polynomials

We have two intervals: $[1, 2]$ and $[2, 3]$, so two cubic polynomials:

$$
S_0(x) = a_0 x^3 + b_0 x^2 + c_0 x + d_0, \quad x \in [1, 2]
$$
$$
S_1(x) = a_1 x^3 + b_1 x^2 + c_1 x + d_1, \quad x \in [2, 3]
$$

There are 8 unknowns: $a_0, b_0, c_0, d_0, a_1, b_1, c_1, d_1$.

##### Step 2: Apply interpolation conditions

The spline must pass through the points:

$$
S_0(1) = 2, \quad S_0(2) = 3
$$
$$
S_1(2) = 3, \quad S_1(3) = 5
$$

This gives 4 equations.

##### Step 3: Apply continuity of first and second derivatives at $x=2$

$$
S_0'(2) = S_1'(2)
$$
$$
S_0''(2) = S_1''(2)
$$

This gives 2 more equations.

##### Step 4: Apply natural boundary conditions

$$
S_0''(1) = 0, \quad S_1''(3) = 0
$$

This gives the final 2 equations.

##### Step 5: Solve the system of 8 equations for the 8 unknowns

Solving these equations yields the coefficients of the two cubic polynomials, which together form the natural cubic spline.


### 5. 📏 General Formulas and Notation for Cubic Splines

To simplify calculations, define:

- $h_j = x_{j+1} - x_j$ (the width of each subinterval)
- $a_j = f(x_j)$ (function values at nodes)

The cubic spline on $[x_j, x_{j+1}]$ can be written as:

$$
S_j(x) = a_j + b_j (x - x_j) + c_j (x - x_j)^2 + d_j (x - x_j)^3
$$

The coefficients satisfy:

- $a_{j+1} = a_j + b_j h_j + c_j h_j^2 + d_j h_j^3$
- $b_{j+1} = b_j + 2 c_j h_j + 3 d_j h_j^2$
- $c_{j+1} = c_j + 3 d_j h_j$

Using these relations and the boundary conditions, a system of linear equations can be formed to solve for the $c_j$ coefficients first, then $b_j$ and $d_j$.


### 6. 🧮 The Linear System for the Second Derivatives

The key step in cubic spline interpolation is solving for the second derivatives at the nodes, which are related to the $c_j$ coefficients.

The system is:

$$
h_{j-1} c_{j-1} + 2(h_{j-1} + h_j) c_j + h_j c_{j+1} = 3 \left( \frac{a_{j+1} - a_j}{h_j} - \frac{a_j - a_{j-1}}{h_{j-1}} \right), \quad j=1, \ldots, n-1
$$

This is a tridiagonal system (only the main diagonal and the two adjacent diagonals have nonzero entries), which can be solved efficiently.

- For **natural splines**, the boundary conditions are $c_0 = 0$ and $c_n = 0$.
- For **clamped splines**, the boundary conditions involve the first derivatives at the endpoints and modify the first and last equations accordingly.


### 7. ✅ Existence and Uniqueness Theorems

- **Theorem 3.0.1 (Natural spline):** For any function $f$ defined on $[a, b]$ with nodes $x_0 < \cdots < x_n$, there exists a unique natural cubic spline interpolant $S$ satisfying $S''(a) = 0$ and $S''(b) = 0$.

- **Theorem 3.0.2 (Clamped spline):** If $f$ is differentiable at the endpoints $a$ and $b$, then there exists a unique clamped cubic spline interpolant $S$ satisfying $S'(a) = f'(a)$ and $S'(b) = f'(b)$.

The uniqueness follows from the fact that the coefficient matrix in the linear system is strictly diagonally dominant, ensuring a unique solution.


### 8. 🧪 Example: Clamped Cubic Spline for $f(x) = e^x$

Given points:

$$
(0, 1), (1, e), (2, e^2), (3, e^3)
$$

and derivatives at endpoints:

$$
f'(0) = 1, \quad f'(3) = e^3
$$

We want to construct a clamped cubic spline $S(x)$ on $[0, 3]$.

- Step sizes: $h_0 = h_1 = h_2 = 1$
- Function values: $a_0 = 1, a_1 = e, a_2 = e^2, a_3 = e^3$

Set up the linear system for $c_j$ using the clamped boundary conditions and solve for $c_0, c_1, c_2, c_3$.

Once $c_j$ are found, calculate $b_j$ and $d_j$ to get the full spline.


### 9. 📉 Error Bound for Cubic Splines

The error between the true function $f$ and the cubic spline $S$ can be bounded if $f$ is sufficiently smooth (has continuous fourth derivative).

**Theorem 3.0.3:**

If $f \in C^4[a,b]$ and

$$
M = \max_{x \in [a,b]} |f^{(4)}(x)|
$$

then for all $x \in [a,b]$,

$$
|f(x) - S(x)| \leq \frac{5}{384} M h^4
$$

where $h = \max_j (x_{j+1} - x_j)$ is the maximum spacing between nodes.

This means the error decreases rapidly as the nodes get closer together (since error depends on $h^4$).


### 10. 📝 Summary

- Cubic spline interpolation fits piecewise cubic polynomials between data points.
- The spline is smooth: continuous function, first derivative, and second derivative.
- Boundary conditions (natural or clamped) are needed to uniquely determine the spline.
- The problem reduces to solving a tridiagonal linear system for the second derivatives.
- Cubic splines provide a smooth, natural-looking curve that approximates the underlying function well.
- Error bounds show that cubic splines are very accurate for smooth functions.



<br>

## Questions

##### 1. What is the primary reason for using cubic spline interpolation instead of a single high-degree polynomial interpolation?  
A) Cubic splines guarantee zero interpolation error.  
B) Cubic splines avoid oscillations common in high-degree polynomials.  
C) Cubic splines require fewer data points than polynomial interpolation.  
D) Cubic splines ensure continuity of the function and its first two derivatives.


##### 2. How many unknown coefficients must be determined when constructing a cubic spline interpolant for $n+1$ data points?  
A) $n$  
B) $2n$  
C) $3n$  
D) $4n$


##### 3. Which of the following conditions must a cubic spline interpolant satisfy at the interior nodes?  
A) The spline values must be equal from both sides.  
B) The first derivatives must be continuous.  
C) The second derivatives must be continuous.  
D) The third derivatives must be continuous.


##### 4. For a natural cubic spline, what are the boundary conditions at the endpoints?  
A) The first derivatives are zero.  
B) The second derivatives are zero.  
C) The function values are zero.  
D) The third derivatives are zero.


##### 5. Which of the following statements about clamped cubic splines is true?  
A) They require knowledge of the function’s derivative at the endpoints.  
B) They always produce less accurate approximations than natural splines.  
C) They impose zero curvature at the endpoints.  
D) They guarantee continuity of the third derivative at the endpoints.


##### 6. Given $n+1$ nodes, how many cubic polynomials make up the cubic spline interpolant?  
A) $n+1$  
B) $n$  
C) $n-1$  
D) $2n$


##### 7. Which of the following is NOT a reason why the coefficient matrix in the cubic spline system is guaranteed to have a unique solution?  
A) The matrix is strictly diagonally dominant.  
B) The matrix is symmetric.  
C) The matrix is positive definite.  
D) The matrix is singular.


##### 8. In the system of equations for cubic splines, the unknowns $c_j$ correspond to which of the following?  
A) The function values at the nodes.  
B) The first derivatives at the nodes.  
C) Half the second derivatives at the nodes.  
D) The third derivatives at the nodes.


##### 9. Which of the following relationships correctly expresses the continuity of the first derivative at an interior node $x_{j+1}$?  
A) $b_j = b_{j+1}$  
B) $b_{j+1} = b_j + 2 c_j h_j + 3 d_j h_j^2$  
C) $c_j = c_{j+1}$  
D) $d_j = d_{j+1}$


##### 10. When constructing a cubic spline, why is it convenient to define $h_j = x_{j+1} - x_j$?  
A) It simplifies the expression of the cubic polynomials.  
B) It reduces the number of unknown coefficients.  
C) It allows the system to be solved without boundary conditions.  
D) It ensures the spline is linear on each subinterval.


##### 11. Which of the following is true about the error bound for cubic spline interpolation with clamped boundary conditions?  
A) The error depends on the maximum fourth derivative of the function.  
B) The error decreases proportionally to $h^2$, where $h$ is the maximum node spacing.  
C) The error decreases proportionally to $h^4$.  
D) The error is independent of the smoothness of the function.


##### 12. For a natural cubic spline, what is the geometric interpretation of the spline curve?  
A) It is the shape a flexible rod would take if forced through the data points.  
B) It is the curve with zero slope at the endpoints.  
C) It is the curve with zero third derivative everywhere.  
D) It is the curve that minimizes the total length between points.


##### 13. Which of the following statements about the continuity of cubic splines is false?  
A) The spline is continuous at all nodes.  
B) The first derivative is continuous at all nodes.  
C) The second derivative is continuous at all nodes.  
D) The third derivative is continuous at all nodes.


##### 14. In the linear system for cubic splines, what role do the boundary conditions play?  
A) They determine the values of $a_j$.  
B) They provide two additional equations to make the system solvable.  
C) They eliminate the need for continuity conditions.  
D) They fix the values of $b_j$ at interior nodes.


##### 15. Suppose you have a cubic spline with clamped boundary conditions but no exact derivative values at the endpoints. What is a likely consequence?  
A) The spline will be identical to the natural spline.  
B) The spline may be less accurate or unstable near the boundaries.  
C) The system of equations will have no solution.  
D) The spline will have discontinuous second derivatives.


##### 16. Which of the following best describes the matrix $A$ in the linear system $Ax = b$ for cubic spline interpolation?  
A) A full dense matrix with no special structure.  
B) A diagonal matrix.  
C) A tridiagonal, strictly diagonally dominant matrix.  
D) A singular matrix.


##### 17. If the nodes $x_j$ are equally spaced, which of the following simplifies in the cubic spline construction?  
A) The matrix $A$ becomes diagonal.  
B) The step sizes $h_j$ are all equal, simplifying the system.  
C) The boundary conditions become unnecessary.  
D) The spline reduces to linear interpolation.


##### 18. Which of the following is a correct statement about the coefficients $b_j$ in the cubic spline polynomial?  
A) They represent the function values at the nodes.  
B) They correspond to the first derivative of the spline at $x_j$.  
C) They are always zero for natural splines.  
D) They are equal to the second derivatives at the nodes.


##### 19. Why is the system of equations for cubic splines typically solved for the $c_j$ coefficients first?  
A) Because $c_j$ are the easiest to compute directly from the data.  
B) Because the system reduces to a tridiagonal system in terms of $c_j$.  
C) Because $c_j$ determine the function values at the nodes.  
D) Because $c_j$ are the boundary conditions.


##### 20. Consider a cubic spline interpolant $S$ for a function $f$ with continuous fourth derivative. Which of the following affects the accuracy of $S$?  
A) The maximum spacing between nodes $h$.  
B) The maximum value of $|f^{(4)}(x)|$ on the interval.  
C) The number of nodes $n$ only, regardless of spacing.  
D) The choice between natural and clamped boundary conditions.



<br>

## Answers

##### 1. What is the primary reason for using cubic spline interpolation instead of a single high-degree polynomial interpolation?  
A) ✗ Cubic splines do not guarantee zero interpolation error.  
B) ✓ Cubic splines avoid oscillations common in high-degree polynomials (Runge’s phenomenon).  
C) ✗ Cubic splines do not necessarily require fewer points; they use the same data points.  
D) ✓ Cubic splines ensure continuity of the function and its first two derivatives, providing smoothness.

**Correct:** B, D


##### 2. How many unknown coefficients must be determined when constructing a cubic spline interpolant for $n+1$ data points?  
A) ✗ Too few; each cubic has 4 coefficients.  
B) ✗ Not enough; 2 per interval is insufficient.  
C) ✗ Still too few.  
D) ✓ Each of the $n$ intervals has 4 coefficients, so total is $4n$.

**Correct:** D


##### 3. Which of the following conditions must a cubic spline interpolant satisfy at the interior nodes?  
A) ✓ The spline values must be equal from both sides (continuity).  
B) ✓ The first derivatives must be continuous.  
C) ✓ The second derivatives must be continuous.  
D) ✗ The third derivatives are generally discontinuous at nodes.

**Correct:** A, B, C


##### 4. For a natural cubic spline, what are the boundary conditions at the endpoints?  
A) ✗ First derivatives are not necessarily zero.  
B) ✓ Second derivatives are zero at endpoints (natural boundary).  
C) ✗ Function values are given by data, not zero.  
D) ✗ Third derivatives are not constrained to zero.

**Correct:** B


##### 5. Which of the following statements about clamped cubic splines is true?  
A) ✓ Clamped splines require knowledge of the function’s derivative at endpoints.  
B) ✗ Clamped splines often produce more accurate approximations, not less.  
C) ✗ Clamped splines do not impose zero curvature; that’s natural splines.  
D) ✗ Third derivatives are not guaranteed continuous at endpoints.

**Correct:** A


##### 6. Given $n+1$ nodes, how many cubic polynomials make up the cubic spline interpolant?  
A) ✗ There are fewer polynomials than nodes.  
B) ✓ There are $n$ intervals, so $n$ cubic polynomials.  
C) ✗ $n-1$ is too few.  
D) ✗ $2n$ is too many.

**Correct:** B


##### 7. Which of the following is NOT a reason why the coefficient matrix in the cubic spline system is guaranteed to have a unique solution?  
A) ✗ Strict diagonal dominance helps guarantee uniqueness.  
B) ✗ The matrix is symmetric, which helps but is not the main reason.  
C) ✗ Positive definiteness also helps guarantee uniqueness.  
D) ✓ The matrix is not singular; singularity would prevent uniqueness.

**Correct:** D


##### 8. In the system of equations for cubic splines, the unknowns $c_j$ correspond to which of the following?  
A) ✗ $a_j$ correspond to function values.  
B) ✗ $b_j$ relate to first derivatives.  
C) ✓ $c_j$ correspond to half the second derivatives at nodes.  
D) ✗ $d_j$ relate to third derivatives.

**Correct:** C


##### 9. Which of the following relationships correctly expresses the continuity of the first derivative at an interior node $x_{j+1}$?  
A) ✗ $b_j = b_{j+1}$ is not generally true.  
B) ✓ $b_{j+1} = b_j + 2 c_j h_j + 3 d_j h_j^2$ expresses derivative continuity.  
C) ✗ $c_j = c_{j+1}$ relates to second derivative continuity but not first derivative.  
D) ✗ $d_j = d_{j+1}$ is not required.

**Correct:** B


##### 10. When constructing a cubic spline, why is it convenient to define $h_j = x_{j+1} - x_j$?  
A) ✓ It simplifies polynomial expressions and system equations.  
B) ✗ It does not reduce the number of unknowns.  
C) ✗ Boundary conditions are still needed.  
D) ✗ The spline is cubic, not linear, on each subinterval.

**Correct:** A


##### 11. Which of the following is true about the error bound for cubic spline interpolation with clamped boundary conditions?  
A) ✓ The error depends on the maximum fourth derivative of the function.  
B) ✗ The error decreases as $h^4$, not $h^2$.  
C) ✓ The error decreases proportionally to $h^4$.  
D) ✗ The error depends on function smoothness.

**Correct:** A, C


##### 12. For a natural cubic spline, what is the geometric interpretation of the spline curve?  
A) ✓ It models the shape of a flexible rod forced through the points.  
B) ✗ The slope at endpoints is not necessarily zero.  
C) ✗ Third derivative is not zero everywhere.  
D) ✗ It does not minimize total length.

**Correct:** A


##### 13. Which of the following statements about the continuity of cubic splines is false?  
A) ✗ The spline is continuous at all nodes.  
B) ✗ The first derivative is continuous at all nodes.  
C) ✗ The second derivative is continuous at all nodes.  
D) ✓ The third derivative is generally discontinuous at nodes.

**Correct:** D


##### 14. In the linear system for cubic splines, what role do the boundary conditions play?  
A) ✗ $a_j$ are fixed by data values, not boundary conditions.  
B) ✓ They provide two additional equations to solve the system uniquely.  
C) ✗ Continuity conditions are still required.  
D) ✗ Boundary conditions do not fix $b_j$ at interior nodes.

**Correct:** B


##### 15. Suppose you have a cubic spline with clamped boundary conditions but no exact derivative values at the endpoints. What is a likely consequence?  
A) ✗ The spline will differ from the natural spline.  
B) ✓ The spline may be less accurate or unstable near boundaries.  
C) ✗ The system still has a solution but may be less reliable.  
D) ✗ Second derivatives remain continuous.

**Correct:** B


##### 16. Which of the following best describes the matrix $A$ in the linear system $Ax = b$ for cubic spline interpolation?  
A) ✗ The matrix is sparse, not full dense.  
B) ✗ It is not diagonal.  
C) ✓ It is tridiagonal and strictly diagonally dominant.  
D) ✗ It is not singular.

**Correct:** C


##### 17. If the nodes $x_j$ are equally spaced, which of the following simplifies in the cubic spline construction?  
A) ✗ The matrix is still tridiagonal, not diagonal.  
B) ✓ Equal $h_j$ simplifies the system coefficients.  
C) ✗ Boundary conditions are still necessary.  
D) ✗ The spline remains cubic, not linear.

**Correct:** B


##### 18. Which of the following is a correct statement about the coefficients $b_j$ in the cubic spline polynomial?  
A) ✗ $a_j$ represent function values.  
B) ✓ $b_j$ correspond to the first derivative of the spline at $x_j$.  
C) ✗ $b_j$ are not zero for natural splines.  
D) ✗ $b_j$ are not equal to second derivatives.

**Correct:** B


##### 19. Why is the system of equations for cubic splines typically solved for the $c_j$ coefficients first?  
A) ✗ $c_j$ are not directly computed from data alone.  
B) ✓ The system reduces to a tridiagonal system in terms of $c_j$, making it easier to solve.  
C) ✗ $c_j$ do not determine function values.  
D) ✗ $c_j$ are not boundary conditions.

**Correct:** B


##### 20. Consider a cubic spline interpolant $S$ for a function $f$ with continuous fourth derivative. Which of the following affects the accuracy of $S$?  
A) ✓ Maximum spacing $h$ affects error magnitude.  
B) ✓ Maximum $|f^{(4)}(x)|$ affects error magnitude.  
C) ✗ Number of nodes alone does not guarantee accuracy without spacing consideration.  
D) ✓ Choice of boundary conditions can affect accuracy.

**Correct:** A, B, D